# SpotifySights

In [1]:
# Libraries needed for the project
import json
import base64
import spotipy                                              # type: ignore                          
import numpy as np
import pandas as pd
import seaborn as sns
from requests import post, get

### API credentials
Remember that you can get the credentials creating an app in the [spotify developers web](https://developer.spotify.com/dashboard).

In [2]:
# Spotify API credentials
SPOTIPY_CLIENT_ID = "4122a22d0e964aa79d115b85f384f661"
SPOTIPY_CLIENT_SECRET = "07f933d0f9cc4dc58d0302c9227d6717"


In [18]:
# Function to get the token
def token():
    """ 
    Function to get the token for the Spotify API, without this the API won't work.
    You don't have to change anything in this function.
    """
    auth_string = f"{SPOTIPY_CLIENT_ID}:{SPOTIPY_CLIENT_SECRET}"
    auth_bytes = auth_string.encode("utf-8")
    auth_encoded = base64.b64encode(auth_bytes)

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": f"Basic {auth_encoded.decode('utf-8')}",
        "Content-Type": "application/x-www-form-urlencoded"
        }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

# Function to get the authorization header
def get_auth_header(token):
    """
    Function to get the authorization header for the Spotify API.
    You don't have to change anything in this function.
    """
    return {"Authorization": "Bearer " + token}

In [19]:
def my_profile():
    """
    Function to get the profile of the user.
    """
    token_ = token()
    print(token_)
    headers = get_auth_header(token_)
    url = "https://api.spotify.com/v1/me"
    result = get(url, headers=headers)
    return json.loads(result.content)

In [20]:

me = my_profile()
print(me)

BQBDfrIXrORjVd8P7plgPiT5er13YESkUld1DsaaGlS-aSaZUyAAi-bKWE3MbrmnHmTiK4appcWuhy7UM2wl98aEokP0mze3z0x2ljsrUoZZDyIknS8
{'error': {'status': 401, 'message': 'Unauthorized.'}}
